In [1]:
import pandas as pd
import json

In [11]:
kdf = pd.read_csv('tracks.csv')

In [3]:
file_paths = [
    'mpd.slice.0-999.json',
    'mpd.slice.99000-99999.json',
    'mpd.slice.995000-995999.json',
    'mpd.slice.996000-996999.json',
    'mpd.slice.997000-997999.json',
    'mpd.slice.998000-998999.json',
    'mpd.slice.999000-999999.json'
]

all_data = []

for file_path in file_paths:
    with open (file_path, 'r') as file:
        data = json.load(file)
        for playlist in data['playlists']:
            playlist_name = playlist['name']
            playlist_desc = playlist.get('description', '')
            for track in playlist['tracks']:
                all_data.append({
                    'playlist_name': playlist_name,
                    'playlist_desc': playlist_desc,
                    'position_in_playlist': track['pos'],
                    'track_name': track['track_name'],
                    'artist_name': track['artist_name'],
                    'duration_ms': track['duration_ms'],
                    'album_name': track['album_name']
                })
xdf = pd.DataFrame(all_data)

In [23]:
xdf

,playlist_name,playlist_desc,position_in_playlist,track_name,artist_name,duration_ms,album_name
0,Throwbacks,,0,Lose Control (feat. Ciara & Fat Man Scoop),Missy Elliott,226863,The Cookbook
1,Throwbacks,,1,Toxic,Britney Spears,198800,In The Zone
2,Throwbacks,,2,Crazy In Love,Beyoncé,235933,Dangerously In Love (Alben für die Ewigkeit)
3,Throwbacks,,3,Rock Your Body,Justin Timberlake,267266,Justified
4,Throwbacks,,4,It Wasn't Me,Shaggy,227600,Hot Shot
...,...,...,...,...,...,...,...
468967,BANDA,,105,La Doble Cara,Banda Carnaval,214706,Las Vueltas De La Vida
468968,BANDA,,106,Adicto a la Tristeza,Edwin Luna y La Trakalosa de Monterrey,198133,La Banda Grande de la Sultana del Norte
468969,BANDA,,107,Mi Nombre Entre Tus Dientes (feat. Big Javy),Edwin Luna y La Trakalosa de Monterrey,208946,La Banda Grande de la Sultana del Norte
468970,BANDA,,108,Como los Gatos (feat. Xitlali Sarmiento),Edwin Luna y La Trakalosa de Monterrey,182093,La Banda Grande de la Sultana del Norte
